# Navie Bayes

朴素贝叶斯分类器。“朴素”：整个形式化过程只做最原始简单的假设.

首先构造一个最简单的概率分类器，然后给出一些假设来学习朴素贝叶斯分类器，借助python将文档切分成“词向量”，然后利用“词向量”对文档进行分类。

## 基于Bayes决策理论的分类方法
**优点**：在数据较少的情况下仍然有效，可以处理多类别问题

**缺点**：对于输入数据的准备方式较为敏感（？）

**适用数据类型**：标称型

## 条件概率

贝叶斯准则告诉我们如何交换条件概率中的条件和结果，即如果已知P(X|C)，要求P(C|X)，那么可以使用下面的计算方法：

$$p(c|x)=\frac{p(x|c)p(c)}{p(x)}$$


## 使用条件概率来分类
